In [1]:
#stby.py
def stby_query(select = ("""SELECT wu.WEBSITE_USER_ID, wu.IS_CLOSED, wu.CREATE_DATE 
                       FROM bdr.WEBSITE_USER wu 
                       WHERE ROWNUM <= 200 """),
               file_name = "stby_query_result.csv",
               write_file = True): 
    """
    
    A function to query the Standby Database that saves the results to a csv file and returns the data in a pandas dataframe.
    Created July 6, 2016 by Jennifer Jones

    Connects to the Standby database: "rptread/rptread@standbydb1.aur.ziprealty.com:1521/stbyntap"
    
    # stby_query(select)
    df = stby_query(select)

    Default inputs:
    ---------------
    select = ("SELECT wu.WEBSITE_USER_ID, wu.IS_CLOSED, wu.CREATE_DATE "
              "FROM bdr.WEBSITE_USER wu "
              "WHERE ROWNUM <= 10 ")
    

    Required modules:
    ----------------
    numpy
    pandas
    time
    cx_Oracle

    """
    
    import numpy as np
    import pandas as pd
    import time
    import cx_Oracle
    
    stby = "rptread/rptread@standbydb2.aur.ziprealty.com:1521/stbypstr"
    
    # connect to the database
    db = cx_Oracle.connect(stby)
    
    # execute the sql select statement
    start = time.time()
    cur = db.cursor()
    cur.arraysize = 100
    cur.execute(select)
    
    col_labels = [x[0] for x in cur.description] 
    select_statement = cur.statement

    # save the result and record how long it took to run the query
    result = cur.fetchall() # or cursor.fetchmany(numRows = nrows) 
    elapsed = (time.time() - start) # uses the 'time' module to measure elapsed time of the query
    
    # close the connection to the database
    cur.close()
    db.close()
    
    print("The query took %.2f seconds (%.2f minutes). \n" %(round(elapsed,3), round(elapsed/60,3)))
    print("The query returned", len(result), "rows. \t shape: ", np.shape(result), "\n")
    
    # Add data to pandas dataframe and return the df
    resultdf = pd.DataFrame(data = result, columns = col_labels)
    
    # Save the results to a csv file to be able to upload it to python at a later date
    #filepath = './stby_query_result.csv'
    if write_file:
        resultdf.to_csv(file_name, sep=',', columns = resultdf.columns, header = True, index = False) #, na_rep="")
    
        print("Results were saved to", file_name, "\n")

    print("The following select statement was executed: \n", select_statement, '\n')
    print("The following columns were returned: \n", col_labels, '\n')
    
    return resultdf

In [7]:
df = stby_query(select = ("""SELECT LISTING_NUM,
       ADDRESS, CITY, ZIP, LATITUDE, LONGITUDE,
       HAS_GARAGE, HAS_CARPORT, PET_RESTRICT, HAS_BRIDGE_VIEW, HAS_MOUNTAIN_VIEW, HAS_CENTRAL_AIR, HAS_HARDWOOD_FLOORS, 
       HAS_BASEMENT, HAS_DEN_OFFICE, HAS_DINING_ROOM, HAS_FAMILY_ROOM,
       BEDROOMS, FULL_BATHS, PARTIAL_BATHS, YEAR_BUILT, STORIES, ROOMS,
       HOME_SQ_FEET, LOT_SIZE, LOT_SQ_FEET, 
       INACT_SELLING_DATE,
       LISTING_AGENT, LISTING_OFFICE,
       SCHOOLDISTRICT, ELEMENTARYSCHOOL, MIDDLESCHOOL, HIGHSCHOOL, ELEM_SCHOOL_DIST, HIGH_SCHOOL_DIST,
       NUM_IMAGES,
       LISTING_PRICE, INACT_SELLING_PRICE
FROM BDR.MLS_CA_BA_INACTIVE 
where INACT_STATUS = 'SOLD' AND
LISTING_DATE > to_date('01-JAN-16') AND
UPPER(COUNTY) = 'ALAMEDA' AND
PROPERTY_TYPE = 'SFR'"""),
                file_name= "alameda.csv", write_file = True)

The query took 12.72 seconds (0.21 minutes). 

The query returned 10587 rows. 	 shape:  (10587, 38) 

Results were saved to alameda.csv 

The following select statement was executed: 
 SELECT LISTING_NUM,
       ADDRESS, CITY, ZIP, LATITUDE, LONGITUDE,
       HAS_GARAGE, HAS_CARPORT, PET_RESTRICT, HAS_BRIDGE_VIEW, HAS_MOUNTAIN_VIEW, HAS_CENTRAL_AIR, HAS_HARDWOOD_FLOORS, 
       HAS_BASEMENT, HAS_DEN_OFFICE, HAS_DINING_ROOM, HAS_FAMILY_ROOM,
       BEDROOMS, FULL_BATHS, PARTIAL_BATHS, YEAR_BUILT, STORIES, ROOMS,
       HOME_SQ_FEET, LOT_SIZE, LOT_SQ_FEET, 
       INACT_SELLING_DATE,
       LISTING_AGENT, LISTING_OFFICE,
       SCHOOLDISTRICT, ELEMENTARYSCHOOL, MIDDLESCHOOL, HIGHSCHOOL, ELEM_SCHOOL_DIST, HIGH_SCHOOL_DIST,
       NUM_IMAGES,
       LISTING_PRICE, INACT_SELLING_PRICE
FROM BDR.MLS_CA_BA_INACTIVE 
where INACT_STATUS = 'SOLD' AND
LISTING_DATE > to_date('01-JAN-16') AND
UPPER(COUNTY) = 'ALAMEDA' AND
PROPERTY_TYPE = 'SFR' 

The following columns were returned: 
 ['LISTING_NUM',

In [4]:
df.describe()

,LATITUDE,LONGITUDE,HAS_GARAGE,HAS_CARPORT,BEDROOMS,FULL_BATHS,PARTIAL_BATHS,YEAR_BUILT,STORIES,ROOMS,HOME_SQ_FEET,LOT_SQ_FEET,NUM_IMAGES,LISTING_PRICE,INACT_SELLING_PRICE
count,10585.000000,10585.000000,10587.0,10587.0,10587.000000,10587.000000,10587.000000,10560.000000,9703.000000,10319.000000,10325.000000,1.051900e+04,10587.000000,1.058700e+04,1.058700e+04
mean,36.330492,-117.629584,0.0,0.0,3.329366,2.057146,0.277416,1959.338826,1.505102,7.802888,1811.033705,9.071533e+04,21.217814,8.247911e+05,8.737005e+05
std,7.059335,22.854786,0.0,0.0,0.910323,0.811337,0.478740,29.599559,0.604346,92.575348,833.521963,4.449581e+06,8.981969,4.075843e+05,4.239976e+05
min,0.000000,-122.307223,0.0,0.0,0.000000,0.000000,0.000000,1870.000000,1.000000,2.000000,420.000000,7.270000e+02,0.000000,1.495000e+05,1.495000e+05
25%,37.628689,-122.195366,0.0,0.0,3.000000,2.000000,0.000000,1939.000000,1.000000,6.000000,1238.000000,4.440000e+03,16.000000,5.780000e+05,6.050000e+05
50%,37.695011,-122.079704,0.0,0.0,3.000000,2.000000,0.000000,1959.000000,1.000000,7.000000,1606.000000,5.702000e+03,24.000000,7.490000e+05,7.820000e+05
75%,37.774071,-121.940948,0.0,0.0,4.000000,2.000000,1.000000,1982.000000,2.000000,8.000000,2150.000000,7.350000e+03,29.000000,9.498000e+05,1.025000e+06
max,37.904289,0.000000,0.0,0.0,8.000000,7.000000,12.000000,2017.000000,3.000000,9408.000000,8639.000000,2.787840e+08,93.000000,6.500000e+06,6.200000e+06


In [5]:
df.shape

(10587, 37)

In [6]:
df.columns

Index(['LISTING_NUM', 'CITY', 'ZIP', 'LATITUDE', 'LONGITUDE', 'HAS_GARAGE',
       'HAS_CARPORT', 'PET_RESTRICT', 'HAS_BRIDGE_VIEW', 'HAS_MOUNTAIN_VIEW',
       'HAS_CENTRAL_AIR', 'HAS_HARDWOOD_FLOORS', 'HAS_BASEMENT',
       'HAS_DEN_OFFICE', 'HAS_DINING_ROOM', 'HAS_FAMILY_ROOM', 'BEDROOMS',
       'FULL_BATHS', 'PARTIAL_BATHS', 'YEAR_BUILT', 'STORIES', 'ROOMS',
       'HOME_SQ_FEET', 'LOT_SIZE', 'LOT_SQ_FEET', 'INACT_SELLING_DATE',
       'LISTING_AGENT', 'LISTING_OFFICE', 'SCHOOLDISTRICT', 'ELEMENTARYSCHOOL',
       'MIDDLESCHOOL', 'HIGHSCHOOL', 'ELEM_SCHOOL_DIST', 'HIGH_SCHOOL_DIST',
       'NUM_IMAGES', 'LISTING_PRICE', 'INACT_SELLING_PRICE'],
      dtype='object')